# Imports

In [ ]:
# Install requirements from requirements file
%pip install -r ../requirements.txt

# Alternative: Install requirements directly
# %pip install blosum
# %pip install Bio
# %pip install torch torchvision torchaudio transformers sentencepiece accelerate --extra-index-url https://download.pytorch.org/whl/cu116
# %pip install protein-bert
# %pip install biopython biotite
# %pip install fair-esm
# %pip install scipy
# %pip install matplotlib
# %pip install seaborn

In [10]:
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
from scipy.stats import t
import blosum as bl
from Bio import SeqIO
import random
from scipy import stats
import torch
import esm
import re
import os
from tqdm import tqdm
import seaborn as sns
import re
import random
import pickle
import statistics
import time
from zipfile import ZipFile
import sys
sys.path.insert(0, "../")

# Importing models and embeddings retrieval methods
from models import get_model
from embeddings import get_fasta_embeddings, get_pair_embeddings, load_fasta

# Retrieve the device (CPU or GPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using device: {}".format(device))

torch.set_grad_enabled(False)

import warnings
warnings.filterwarnings("ignore")

# Available models
MODELS_LIST = ["ProtT5", "ProtBert", "ProtAlbert", "ProtXLNet", "ESM1b", "ESM2"]

# Available alignment types
ALIGNMENT_TYPES = ["Global-regular" , "Global-end-gap-free"]

c:\Users\Riley\iCloudDrive\Repositories\e-score\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
Using device: cpu
Using device: cpu


# Alignment Algorithms

## Global

In [11]:
def affine_global_dp(seq_1, seq_2, g_open, g_ext,
                     scoring="ProtT5", Model=None, Model_tokenizer=None):
    """ Performs global alignment of two sequences using affine gap penalty dynamic programming algorithm """
    m, n = len(seq_1), len(seq_2)

    # Initialize matrices
    M = np.zeros([m + 1, n + 1])
    M[0, 1:] = g_open + g_ext * np.arange(0, n, 1)
    M[1:, 0] = g_open + g_ext * np.arange(0, m, 1)

    # Copies to avoid Gotoh's error
    L, U = np.copy(M), np.copy(M)
    L[1:, 0] += g_open;
    U[0, 1:] += g_open

    # Tracer matrix
    tracer = np.zeros([np.shape(M)[0], np.shape(M)[1], 7])
  
    # Get embeddings
    emb1, emb2 = get_pair_embeddings(seq_1, seq_2, Model, Model_tokenizer, scoring)
    cos = torch.nn.CosineSimilarity(dim=0)

    # Dynamic programming algorithm
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            l_arr = np.array([M[i, j - 1] + g_open, L[i, j - 1] + g_ext])
            L[i, j] = np.max(l_arr)
            l_where = l_arr == np.max(l_arr)

            u_arr = np.array([M[i - 1, j] + g_open, U[i - 1, j] + g_ext])
            U[i, j] = np.max(u_arr)
            u_where = u_arr == np.max(u_arr)

            if scoring in MODELS_LIST:
                sim = cos(torch.tensor(emb1[i - 1], dtype=torch.float32), 
                          torch.tensor(emb2[j - 1], dtype=torch.float32)).item()
                m_arr = np.array([M[i - 1, j - 1] + sim, U[i, j], L[i, j]])

            M[i, j] = np.max(m_arr)
            m_where = m_arr == np.max(m_arr)

            idx = np.hstack([m_where, u_where, l_where])
            tracer[i, j, idx] = 1

    # Traceback
    alignment = traceback_g(tracer, seq_1, seq_2, affine=True)
    alignment = list(set(map(tuple, alignment)))

    return M, L, U, tracer, alignment


def traceback_g(tracer, seq_1, seq_2, mat=None, affine=False, roadmap=0):
    """ Performs traceback step of the sequence alignment algorithm - reconstructs aligned sequences from the tracer matrix 
    Based on Smith-Waterman algorithm
    """
    # Get sequence lengths
    m, n = len(seq_1), len(seq_2)

    # Convert to numpy arrays
    x = np.array(list(seq_1), dtype='object')
    y = np.array(list(seq_2), dtype='object')

    # Set start location
    st = [m + 1, n + 1]
    st_lv = 0  # Start in midgard

    while st[0] > 1 and st[1] > 1:
        B = np.zeros([2, 2])  # define 2x2 box which specifies which way to move

        Tr_size = 7 if affine else 3  # Size of tracer array
        Tr = np.zeros([Tr_size])  # define a Tr_size x 1 Tr array (will store arrows at each step)

        # Get tracer array
        for i in range(Tr_size):
            Tr[i] = np.copy(tracer[st[0] - 1, st[1] - 1, i])

        # Bifurcations
        levels = [[2, 0, 1], [4, 3], [6, 5]] if affine else [[2, 0, 1]]

        for l in levels:
            if np.sum(Tr[l]) > 1:
                choose = np.where(Tr[l] == 1)[0]
                Tr[l] = 0
                r = np.random.choice(choose, 1)[0] if roadmap == 0 else choose[-1] if roadmap == 1 else choose[0]
                if roadmap not in [0, 1, 2]:
                    raise Exception("roadmap only accepts 0: random turning, 1: highroad, 2: lowroad")
                Tr[l[r]] = 1

        # Diagonal
        if Tr[0] == 1 and st_lv == 0:
            B[0, 0] = 1

        # End gaps
        if Tr[1] == 1 and st_lv == 0:
            if affine:
                st_lv = 1  # level up
            else:
                B[0, 1] = 1

        if Tr[2] == 1 and st_lv == 0:
            if affine:
                st_lv = 2  # level down
            else:
                B[1, 0] = 1

        # Affine gaps allow for level shifts
        if affine:
            if Tr[4] == 1 and st_lv == 1:  # move up
                B[0, 1] = 1

            if Tr[3] == 1 and st_lv == 1:  # move up back to main
                st_lv = 0
                B[0, 1] = 1

            if Tr[6] == 1 and st_lv == 2:  # move left
                B[1, 0] = 1

            if Tr[5] == 1 and st_lv == 2:  # move left back to main
                st_lv = 0
                B[1, 0] = 1

        # Movements
        if B[0, 1] == 1:  # upward
            y = np.insert(y, st[1] - 1, '-')  # add a gap
            st[0] -= 1

        if B[1, 0] == 1:  # leftward
            x = np.insert(x, st[0] - 1, '-')  # add a gap
            st[1] -= 1

        if B[0, 0] == 1:  # diagonal
            st[1] -= 1
            st[0] -= 1

    # Some end gaps are left when you hit the upper/lower end of the matrix or a 0
    end_size = np.size(x) - np.size(y)  # how many gaps and for which sequence
    end_gap = ['-'] * abs(end_size)
    if end_size > 0:
        y = np.insert(y, 0, end_gap)
    elif end_size < 0:
        x = np.insert(x, 0, end_gap)

    # Check no overlapping gaps
    x = np.where(((x == '-') & (y == '-')), None, x)
    y = np.where((x == None), '', y)
    x = np.where((x == None), '', x)

    return np.sum(x), np.sum(y)

## Prefix/Suffix

In [12]:
def affine_semi_global_dp(seq_1, seq_2, g_open, g_ext,
                          high_low=False, scoring="ProtT5", Model=None, Model_tokenizer=None):
    """ Performs semi-global alignment of two sequences using affine gap penalty dynamic programming algorithm """
    m, n = len(seq_1), len(seq_2)
    
    # Initialize matrices
    M = np.zeros([m + 1, n + 1])
    L, U = np.copy(M), np.copy(M)

    # Tracer matrix
    tracer = np.zeros([np.shape(M)[0], np.shape(M)[1], 7])

    # Get embeddings
    emb1, emb2 = get_pair_embeddings(seq_1, seq_2, Model, Model_tokenizer, scoring)
    cos = torch.nn.CosineSimilarity(dim=0)

    # Dynamic programming algorithm
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            l_arr = np.array([M[i, j - 1] + g_open, L[i, j - 1] + g_ext])
            L[i, j] = np.max(l_arr)
            l_where = l_arr == np.max(l_arr)

            u_arr = np.array([M[i - 1, j] + g_open, U[i - 1, j] + g_ext])
            U[i, j] = np.max(u_arr)
            u_where = u_arr == np.max(u_arr)

            if scoring in MODELS_LIST:
                sim = cos(torch.tensor(emb1[i - 1], dtype=torch.float32), 
                          torch.tensor(emb2[j - 1], dtype=torch.float32)).item()
                m_arr = np.array([M[i - 1, j - 1] + sim, U[i, j], L[i, j]])

            M[i, j] = np.max(m_arr)
            m_where = m_arr == np.max(m_arr)

            idx = np.hstack([m_where, u_where, l_where])
            tracer[i, j, idx] = 1

    alignment = [traceback_sg(tracer, seq_1, seq_2, mat=M, affine=True, local= True, roadmap=0)]
    alignment = list(set(map(tuple, alignment)))

    return M, L, U, tracer, alignment


def traceback_sg(tracer, seq_1, seq_2, mat=None, local=False, affine=False, roadmap=0):
    """ Performs traceback step of the sequence alignment algorithm - reconstructs aligned sequences from the tracer matrix """
    m, n = len(seq_1), len(seq_2)

    # Convert to numpy arrays
    x = np.array(list(seq_1), dtype='object')
    y = np.array(list(seq_2), dtype='object')

    # Set start location (random maxima if 0)
    r = np.random.choice(range(np.size(np.where(mat == np.max(mat))[0])), 1)[0] if roadmap == 0 else -1 if roadmap == 1 else 0
    st = [(np.where(mat == np.max(mat))[0][r]) + 1, (np.where(mat == np.max(mat))[1][r]) + 1]

    # Set starting gaps based on the start location
    start_size = (m - st[0]) - (n - st[1])  # how many gaps and for which sequence
    start_gap = (['-'] * abs(start_size))
    if start_size > 0:
        y = np.append(y, start_gap)
    elif start_size < 0:
        x = np.append(x, start_gap)

    st_lv = 0  # start in midgard

    while st[0] > 1 and st[1] > 1:
        B = np.zeros([2, 2])  # define 2x2 box which specifies which way to move

        Tr_size = 7 if affine else 3  # size of tracer array
        Tr = np.zeros([Tr_size])  # define a Tr_size x 1 Tr array (will store arrows at each step)

        # Tracer
        for i in range(Tr_size):
            Tr[i] = np.copy(tracer[st[0] - 1, st[1] - 1, i])

        # Bifurcations
        levels = [[2, 0, 1], [4, 3], [6, 5]] if affine else [[2, 0, 1]]

        for l in levels:
            if np.sum(Tr[l]) > 1:
                choose = np.where(Tr[l] == 1)[0]
                Tr[l] = 0
                # 0: random turning, 1: highroad, 2: lowroad
                r = np.random.choice(choose, 1)[0] if roadmap == 0 else choose[-1] if roadmap == 1 else choose[0]
                if roadmap not in [0, 1, 2]:
                    raise Exception("roadmap only accepts 0: random turning, 1: highroad, 2: lowroad")
                
                Tr[l[r]] = 1

        # level up-down
        if Tr[0] == 1 and st_lv == 0:  # diagonal
            B[0, 0] = 1

        if Tr[1] == 1 and st_lv == 0:
            if affine:
                st_lv = 1  # level up
            else:
                B[0, 1] = 1

        if Tr[2] == 1 and st_lv == 0:
            if affine:
                st_lv = 2  # level down
            else:
                B[1, 0] = 1

        # Affine gaps allow for level shifts
        if affine:
            if Tr[4] == 1 and st_lv == 1:  # move up
                B[0, 1] = 1

            if Tr[3] == 1 and st_lv == 1:  # move up back to main
                st_lv = 0
                B[0, 1] = 1

            if Tr[6] == 1 and st_lv == 2:  # move left
                B[1, 0] = 1

            if Tr[5] == 1 and st_lv == 2:  # move left back to main
                st_lv = 0
                B[1, 0] = 1

        if local and mat[st[0] - 1, st[1] - 1] == 0:
            break

        # Movements
        if B[0, 1] == 1:  # upward
            y = np.insert(y, st[1] - 1, '-')  # add a gap
            st[0] -= 1

        if B[1, 0] == 1:  # leftward
            x = np.insert(x, st[0] - 1, '-')  # add a gap
            st[1] -= 1

        if B[0, 0] == 1:  # diagonal
            st[1] -= 1
            st[0] -= 1

    # Some end gaps are left when you hit the upper/lower end of the matrix or a 0
    end_size = np.size(x) - np.size(y)  # how many gaps and for which sequence
    end_gap = (['-'] * abs(end_size))
    if end_size > 0:
        y = np.insert(y, 0, end_gap)
    elif end_size < 0:
        x = np.insert(x, 0, end_gap)

    # Check no overlapping gaps
    x = np.where(((x == '-') & (y == '-')), None, x)
    y = np.where((x == None), '', y)
    x = np.where((x == None), '', x)

    return np.sum(x), np.sum(y)

# Aux Funx

In [13]:
def get_sequences_only(seqs):
    """ Extracts a list of (sequence) from a list of (name, sequence) tuples """
    return [seq[1] for seq in seqs]

def aligned_to_indexed(seqs):
  """ Removes dashes (-) in a sequence and creates positions array for non-dash residues """
  # Replace dashes with empty string
  no_dash = [seq.replace("-", "") for seq in seqs]

  # Create positions array for non-dash residues
  positions = [[i for i, char in enumerate(seq) if char != "-"] for seq in seqs]

  return no_dash, positions

def length_matcher(x , y , place = ""):
  """ Matches the length between x and y with spaces if necessary """
  length = 5

  def add_spaces(value):
    spaces = abs(len(value) - length)
    return " " * spaces + value if place == "Back" else value + " " * spaces

  return add_spaces(x), add_spaces(y)

# Alignment Computations

In [14]:
def get_alignments(prot1, prot2, gap_penalty = 0, gap_extension_penalty = 0,
                   scoring = "ProtT5", alignment_type = "Global-regular", Model = "", Model_Tokenizer = ""):
  """ Gets the alignments between two sequences """
  
  if alignment_type == "Global-regular":
    M, L, U, tracer, alignment = affine_global_dp(prot1, prot2, gap_penalty, gap_extension_penalty, 
                                                  scoring = scoring , Model = Model, Model_tokenizer = Model_Tokenizer)
    max_score = np.max(M)

  if alignment_type == "Global-end-gap-free" or alignment_type == "End-Gap-Free":
    M, L, U, tracer, alignment = affine_semi_global_dp(prot1, prot2, gap_penalty, gap_extension_penalty, 
                                                        scoring = scoring , Model = Model, Model_tokenizer = Model_Tokenizer)
    max_score = max(M[-1,-1],L[-1,-1],U[-1,-1])

  # Return (reference alignment, query alignment, alignment score)
  aligned1 = alignment[0][0]
  aligned2 = alignment[0][1]

  return aligned1, aligned2, max_score

In [15]:
def get_visualization(prot1, prot2, score, Type = "", Model = "", Model_Tokenizer = ""):
  MODELS_LIST = ["ProtT5" , "ProtBert" , "ProtAlbert" , "ProtXLNet" , "ESM1b" , "ESM2"]
  cos = torch.nn.CosineSimilarity(dim=0)

  seqs = [prot1 , prot2]
  no_dash, positions = aligned_to_indexed(seqs)

  # Get embeddings
  p1_emb, p2_emb = get_pair_embeddings(no_dash[0], no_dash[1], Model, Model_Tokenizer, Type)

  p1_revived = ""
  p2_revived = ""
  aligned_info = ""

  for i in range(len(prot1)):
    if i in positions[0]:
      p1_revived += prot1[i]
    else:
      p1_revived += "-"

    if i in positions[1]:
      p2_revived += prot2[i]
    else:
      p2_revived += "-"


    if p1_revived[-1] == p2_revived[-1]:
      aligned_info += p1_revived[-1]
    elif p1_revived[-1] == "-" or p2_revived[-1] == "-":
      aligned_info += " "
    elif p1_revived[-1] != p2_revived[-1]:
      if Type in MODELS_LIST:
        sim = cos(torch.tensor(p1_emb[0][positions[0].index(i)] , dtype = torch.float32) ,
                  torch.tensor(p2_emb[0][positions[1].index(i)] , dtype = torch.float32)).item()
        aligned_info += " "

  del model
  del tokenizer

  return p1_revived, aligned_info, p2_revived, score

# Alignment For 2 Sequences

In [16]:
def alignment_file_TXT(saving_add, seqs_path, scoring, alignment_type,
                      gap_penalty, gap_extension_penalty):
  """ Creates and outputs the alignment file between two sequences """
  print(f"Device: {device}")

  # Get selected model
  Model , Model_Tokenizer = get_model(scoring)

  # Load sequences from FASTA file
  seqs = load_fasta(seqs_path)

  # Get protein sequences
  prot1 = seqs[0][1]
  prot2 = seqs[1][1]

  # Get names of protein sequences
  name1 = seqs[0][0]
  name2 = seqs[1][0]

  # Get alignments and visualization
  reference_al, query_al, al_score = get_alignments(prot1, prot2, gap_penalty = gap_penalty,
                                              gap_extension_penalty = gap_extension_penalty ,
                                              scoring = scoring , alignment_type = alignment_type,
                                              Model = Model , Model_Tokenizer = Model_Tokenizer)

  p1_al , aligned_info , p2_al , al_score = get_visualization(reference_al , query_al, al_score , Type = scoring,
                                                              Model = Model, Model_Tokenizer = Model_Tokenizer)

  if not os.path.exists(saving_add):
    os.makedirs(saving_add)

  file_name = saving_add + seqs_path.split("/")[-1].split(".")[-2] + "_" + scoring + "_" + alignment_type + "_"
  file_name += str(gap_penalty) + "_" + str(gap_extension_penalty) + "_"+ "Alignment" + ".txt"
  f = open(file_name, "w")

  # Write Sequence 1 Information
  f.write("Seq 1 \n")
  f.write(">" + name1 + "\n")
  f.write(reference_al.replace("-" , "") + "\n")
    
  # Write Sequence 2 Information
  f.write("Seq 2 \n")
  f.write(">" + name2 + "\n")
  f.write(query_al.replace("-" , "") + "\n\n")
    
  # Write Alignment Information
  f.write("Alignment Type : " + alignment_type + "\n\n")
  f.write("Opening Gap Penalty : " + str(gap_penalty) + "\n")
  f.write("Extension Gap Penalty : " + str(gap_extension_penalty) + "\n")
  f.write("Scoring System : " + scoring + "\n")
  f.write("Score : "  + str(al_score) + "\n\n")

  p1_pos = 1
  p2_pos = 1
  aligned_gaps = ""

  for j in range(int(len(p1_al) / 60) + 1):
    p1_posix = p1_al[j * 60: (j + 1) * 60]
    p2_posix = p2_al[j * 60: (j + 1) * 60]
    p1_back_str, p2_back_str = length_matcher(str(p1_pos) , str(p2_pos) , place = "Front")

    for k in range(len(p1_posix)):
      if p1_posix[k] != "-":
        p1_pos += 1
      if p2_posix[k] != "-":
        p2_pos += 1

    p1_end_str, p2_end_str = length_matcher(str(p1_pos - 1) , str(p2_pos - 1) , place = "Back")
    aligned_gaps = " " * len(p1_back_str)

    f.write("Seq 1 : " + p1_back_str + " " + p1_al[j * 60: (j + 1) * 60] + " " + p1_end_str + "\n")
    f.write("        "  +  aligned_gaps + " " + aligned_info[j * 60: (j + 1) * 60] + "\n")
    f.write("Seq 2 : "  + p2_back_str + " " + p2_al[j * 60: (j + 1) * 60] + " " + p2_end_str + "\n\n")

  print("Alignment Computation is Done!")
  del Model
  del Model_Tokenizer

# Cosine Similarity

In [22]:
def get_cosine_similarities(seq_1, seq_2, model, tokenizer, scoring):
    """ Gets the cosine similarity between two sequences """
    cos = torch.nn.CosineSimilarity(dim=0)

    # Get embeddings
    emb1, emb2 = get_pair_embeddings(seq_1, seq_2, model, tokenizer, scoring)

    m, n = len(seq_1), len(seq_2)

    cosine_similarities = np.zeros([m, n])

    for i in range(m):
        for j in range(n):
            sim = cos(torch.tensor(emb1[i], dtype=torch.float32), 
                      torch.tensor(emb2[j], dtype=torch.float32)).item()
            cosine_similarities[i, j] = sim
            
    return cosine_similarities

SyntaxError: incomplete input (1780860152.py, line 13)

## Guide

In [17]:
def user_guide(MODELS_LIST):
  """ User guide for the E-score program """
  print("Parameters & Descriptions:")
  print("  saving_add:".ljust(25) + "Output directory path")
  print("  seqs_path:".ljust(25) + "FASTA file with two protein sequences")
    
  print("  scoring_type:".ljust(25) + "Model for embedding production (", end = "")
  for model_name in MODELS_LIST[:-1] : print(model_name + ", " , end = "")
  print(MODELS_LIST[-1] + ")")

  print("  alignment_type:".ljust(25) + "Global-regular or Global-end-gap-free")
  print("  gap_penalty:".ljust(25) + "Default: -1.0 | Recommended Values: -4.0, -3.0, -2.0, -1.5, -1.0, -0.5")
  print("  gap_extension_penalty:".ljust(25) + "Default: -0.2 | Recommended Values: -1.0, -0.8, -0.5, -0.3, -0.2, -0.1")

# Tests

In [18]:
user_guide(MODELS_LIST)

Parameters & Descriptions:
  saving_add:            Output directory path
  seqs_path:             FASTA file with two protein sequences
  scoring_type:          Model for embedding production (ProtT5, ProtBert, ProtAlbert, ProtXLNet, ESM1b, ESM2)
  alignment_type:        Global-regular or Global-end-gap-free
  gap_penalty:           Default: -1.0 | Recommended Values: -4.0, -3.0, -2.0, -1.5, -1.0, -0.5
  gap_extension_penalty: Default: -0.2 | Recommended Values: -1.0, -0.8, -0.5, -0.3, -0.2, -0.1


In [19]:
# Defining Parameters
saving_add =  "./content/"
seqs_path = "../data/Test2.fasta"
scoring = MODELS_LIST[0]
alignment_type = ALIGNMENT_TYPES[0]
gap_penalty = -1
gap_extension_penalty = -0.2

# Generating Alignment File
# alignment_file_TXT(saving_add = saving_add , seqs_path = seqs_path, scoring = scoring, alignment_type = alignment_type, gap_penalty = gap_penalty, gap_extension_penalty = gap_extension_penalty)

Device: cpu
Initializing ProtT5


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Generating T5 Embeddings
Generating T5 Embeddings
Alignment Computation is Done!


# Analysis

## Embeddings

In [24]:
def analyze_embeddings(seqs_path, model, tokenizer, scoring):
    """ Analyze the embeddings between two sequences """
    embeddings = get_fasta_embeddings(seqs_path, model, tokenizer, scoring)
    return embeddings

In [25]:
# Parameters
saving_add = "./analysis/"
seqs_path = "../data/Test2.fasta"
scoring = MODELS_LIST[0] # ProtT5
alignment_type = ALIGNMENT_TYPES[0] # Global

Model, Tokenizer = get_model(scoring)

# Analysis
print(analyze_embeddings(seqs_path, Model, Tokenizer, scoring))

Initializing ProtT5
Generating T5 Embeddings
